In [ ]:
!pip3 install pycuda

     |████████████████████████████████| 1.6MB 4.9MB/s 
     |████████████████████████████████| 61kB 17.0MB/s 
     |████████████████████████████████| 471kB 48.4MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4536128 sha256=4ceab1c040ddaa7a010f6cf3bfd98f59152d14fcf88e0c184748d43cc45bc2af
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for pytools: filename=pytools-2019.1.1-py2.py3-none-any.whl size=58424 sha256=8c0b901deac1d7b337b9744903554ad532444c81f840552e1dd778a360f8d804
  Stored in directory: /root/.cache/pip/wheels/83/df/0b/75ac4572aaa93e3eba6a58472635d0fda907f5f4cf884a3a0c
  Created wheel for mako: filename=Mako-1.1.0-cp36-none-any.whl size=75363 sha256=d538c2ee0a69710a5c15de872c70c32518a4e8d8a5d96955d41243352163ff7c
  Stored in directory: /root/.cache/pip/wheels/98/32/7b/a291926643fc1d1e02593e0d9e247c5a866a366b8343b7aa27
Successfully built pycuda pytools mako


In [ ]:
import math
import numpy as np
import pycuda.gpuarray as gpuarray
from pycuda.compiler import DynamicSourceModule
import pycuda.autoinit

In [ ]:
module = DynamicSourceModule("""
__global__ void add_two_vector(int nx, float *arr1, float *arr2, float *res){
    int x = threadIdx.x + blockDim.x * blockIdx.x;
    if (x < nx){
        res[x] = arr1[x] + arr2[x];
    }
}
__global__ void add_two_vector_dynamic(int *grid, int *block, int nx, float *arr1, float *arr2, float *res){
dim3 grid_ = dim3(grid[0], grid[1], grid[2]);
dim3 block_ = dim3(block[0], block[1], block[2]);
add_two_vector<<<grid_, block_>>>(nx, arr1, arr2, res);
}
""")

In [ ]:
add_two_vector_dynamic = module.get_function("add_two_vector_dynamic")

In [ ]:
num_comp = np.int32(10)
arr1 = np.arange(num_comp, dtype=np.float32)
arr2 = np.arange(num_comp, dtype=np.float32)

In [ ]:
np.random.shuffle(arr2)

In [ ]:
res_gpu = gpuarray.zeros(num_comp, dtype=np.float32)

In [ ]:
threads_per_block = (256, 1, 1)
blocks_per_grid = (math.ceil(num_comp / threads_per_block[0]), 1, 1)

In [ ]:
block = np.array(threads_per_block, dtype=np.int32)
grid = np.array(blocks_per_grid, dtype=np.int32)

In [ ]:
arr1_gpu = gpuarray.to_gpu(arr1)
arr2_gpu = gpuarray.to_gpu(arr2)
block_gpu = gpuarray.to_gpu(block)
grid_gpu = gpuarray.to_gpu(grid)

In [ ]:
add_two_vector_dynamic(grid_gpu, block_gpu, num_comp, arr1_gpu, arr2_gpu, res_gpu, block=(1, 1, 1), grid=(1, 1, 1))

In [ ]:
res_gpu.get()

array([ 7.,  5.,  5.,  9.,  6., 14., 14., 12.,  9.,  9.], dtype=float32)

In [ ]:
arr1 + arr2

array([ 7.,  5.,  5.,  9.,  6., 14., 14., 12.,  9.,  9.], dtype=float32)